In [1]:
from citableclass import Credentials, Citable, Json2DF

/mnt/ResearchCloud/Projects/geography/.kernel/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook, output_file, save
output_notebook()
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, WheelZoomTool
from bokeh.layouts import row, gridplot, layout

Loading BokehJS ...

In [4]:
title = Credentials(authors='olivier.defaux,gerd.grasshoff,malte.vogl', project='geography',title='Maps of the Iberian Peninsula')
title.titlepage()

On the basis of both recensions of the Geography, the localities of the Iberian peninsula are plotted and compared.

### Load resources

In [5]:
cite = Citable('dict_omega.json',formats='local', project='geography')
x1= cite.digitalresource()

cite = Citable('dict_xi.json',formats='local', project='geography')
x2= cite.digitalresource()

### Transforming to multi-level dataframes

In [6]:
init = Json2DF(x1,['book_ID','chap_ID','sec_ID','ID'],level=4)
init.convert()
dfOmega = init.dataframe

init2 = Json2DF(x2,['book_ID','chap_ID','sec_ID','ID'],level=4)
init2.convert()
dfXi = init2.dataframe

### Translate greek to decimal coordinates

In [7]:
def reformatCoord(row,longLat, xy='coord_x'):
    if type(row[xy]) == dict:
        return (row[xy][longLat]['integer'].strip(),row[xy][longLat]['fraction'].strip())
    else:
        return False

In [8]:
gfrac={"":0,"Lς":2/3,"ςL":2/3,"ιβ":1/12,"ς":1/6,"δ":1/4,"γ":1/3,"γιβ":5/12,"L":1/2,"Lιβ":7/12,"γο":2/3,"Lδ":3/4,"Lγ":5/6,"Lγιβ":11/12,"η":1/8}
gint={"":0,"α":1,"β":2,"γ":3,"δ":4,"ε":5,"ς":6,"ζ":7,"η":8,"θ":9,"ι":10,"κ":20,"λ":30,"μ":40}

In [9]:
def translate(row):
    if type(row)==tuple:
        try:
            temp_frac = gfrac[row[1]]
        except:
            temp_frac = gint[row[1]]
        try:
            if len(row[0]) == 1:
                temp_int = gint[row[0]]
            elif len(row[0]) == 2:
                temp_int = gint[row[0][0]] + gint[row[0][1]]
            elif row[0] == '':
                temp_int = 0
        except:
            temp_int = None
        return temp_int + temp_frac

Omega recension

In [10]:
dfTemp = dfOmega.copy()
dfTemp['longitude_Omega'] = dfOmega.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(translate)
dfTemp['latitude_Omega'] = dfOmega.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(translate)

Xi recension

In [11]:
dfTempX = dfXi.copy()
dfTempX['longitude_Xi'] = dfXi.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(translate)
dfTempX['latitude_Xi'] = dfXi.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(translate)

# Data preparation for the map drawing

### Coasts and boundaries (Omega)

Selection of coastal localities

In [12]:
f = dfTemp[(dfTemp.type_sec == 'coast section') & (dfTemp.type == 'locality') & dfTemp.category.apply(lambda row: row not in ['boundary','river path','river source'])][['longitude_Omega','latitude_Omega']]

Coastal lines

In [13]:
#fbae = coast of Baetica, omega
baria = dfTemp[dfTemp.toponym == 'ἐφʹ ἧς Βαρεία πόλις '][['longitude_Omega','latitude_Omega']]
fbae = f.iloc[0:28]
fbae = fbae.append(baria)
fbae = fbae.values

In [14]:
#flus = coast of Lusitania, omega
flus = f.iloc[28:42].append(f.iloc[0:1]).sort_index(ascending=True)
flus = flus.values

In [15]:
#ftar = coast of Tarraconensis, omega
ftar1 = f.iloc[42:70].append(f.iloc[41:42]).sort_index(ascending=True)
ftar1 = ftar1.values
ftar2 = f.iloc[70:].sort_index(ascending=False)
baebound = dfTemp[dfTemp.toponym == 'μετὰ τὸ πρὸς τῇ Βαιτικῇ πέρας'][['longitude_Omega','latitude_Omega']]
ftar2 = ftar2.append(baebound).append(baria)
ftar2 = ftar2.values

### Coasts and boundaries (Xi)

Selection of coastal localities

In [16]:
g = dfTempX[(dfTempX.type_sec == 'coast section') & (dfTempX.type == 'locality') & dfTempX.category.apply(lambda row: row not in ['boundary','river path','river source'])][['longitude_Xi','latitude_Xi']]

Coastal lines

In [17]:
#gbae = coast of Baetica, xi
bariaX = dfTempX[dfTempX.toponym == 'ἐφʹ ἧς Βαραλία πόλις'][['longitude_Xi','latitude_Xi']]
gbae = g.iloc[0:26]
gbae = gbae.append(bariaX)
gbae = gbae.values

In [18]:
#glus = coast of Lusitania, xi
glus = g.iloc[27:41].append(g.iloc[0:1]).sort_index(ascending=True)
glus = glus.values

In [19]:
#gtar = coast of Tarraconensis, xi
gtar1 = g.iloc[41:68].append(g.iloc[40:41]).sort_index(ascending=True)
gtar1 = gtar1.values
gtar2 = g.iloc[69:].sort_index(ascending=False)
baeboundX = dfTempX[dfTempX.toponym == 'μετὰ τὸ πρὸς τῇ Βαιτικῇ πέρας'][['longitude_Xi','latitude_Xi']]
gtar2 = gtar2.append(baeboundX).append(bariaX)
gtar2 = gtar2.values

# Maps

### Omega recension

In [20]:
p = figure(title='Omega Recension', width=1000, height=800, x_range=(1.5, 22), y_range=(35.5, 47), tools= [BoxZoomTool(), WheelZoomTool(), ResetTool()])

# coasts and boundaries
p.line(fbae[:,0],fbae[:,1], color='grey')
p.line(flus[:,0],flus[:,1], color='grey')
p.line(ftar1[:,0],ftar1[:,1], color='grey')
p.line(ftar2[:,0],ftar2[:,1], color='grey')
p.line([4.083,6.333,9,12], [37.667,39,39,37.25], line_alpha=0.8, color='grey')
p.line([5.333,9.167,9,9], [41.833,41.333,40.167,39], line_alpha=0.8, color='grey')
p.line([15,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color='grey')

# localities
p.circle(np.array(dfTemp['longitude_Omega']),np.array(dfTemp['latitude_Omega']), fill_color='blue',size=6, fill_alpha=.9, line_color='blue',line_alpha=0.7)

show(p);

### Xi recension

In [21]:
q = figure(title='Xi Recension', width=1000, height=800, x_range=(1.5, 22), y_range=(35.5, 47), tools= [BoxZoomTool(), WheelZoomTool(), ResetTool()])

# coasts and boundaries
q.line(gbae[:,0],gbae[:,1], color='grey')
q.line(glus[:,0],glus[:,1], color='grey')
q.line(gtar1[:,0],gtar1[:,1], color='grey')
q.line(gtar2[:,0],gtar2[:,1], color='grey')
q.line([4.083,6.333,9,12], [37.667,39,39,37.25], line_alpha=0.8, color='grey')
q.line([5.333,9.333,9,9], [41.833,41.833,40.5,39], line_alpha=0.8, color='grey')
q.line([15.167,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color='grey')

#localities
q.circle(np.array(dfTempX['longitude_Xi']),np.array(dfTempX['latitude_Xi']), fill_color='red',size=6, fill_alpha=.8, line_color='red',line_alpha=0.6)

show(q)

## Comparison Ξ and Ω

In [22]:
a=dfTempX.reset_index(level=['chap_ID','sec_ID','ID'])
a=a[a.longitude_Xi.apply(lambda row: row not in [0])]
a=a[['ID','longitude_Xi','latitude_Xi']].dropna()

b=dfTemp.reset_index(level=['chap_ID','sec_ID','ID'])
b=b[b.longitude_Omega.apply(lambda row: row not in [0])]
b=b[['ID','longitude_Omega','latitude_Omega']].dropna()

c = pd.merge(left=a, right=b,how='inner')

IbEq = c[c["longitude_Xi"]==c["longitude_Omega"]]
IbEq = IbEq[IbEq["latitude_Xi"]==IbEq["latitude_Omega"]]

In [23]:
r = figure(title='Comparison between Xi (red) and Omega (blue)', width=1000, height=800, x_range=(1.5, 22), y_range=(35.5, 47), tools= [BoxZoomTool(), WheelZoomTool(), ResetTool()])

r.circle(np.array(IbEq["longitude_Xi"]),np.array(IbEq["latitude_Xi"]), size=5.5,fill_color='lightgrey', fill_alpha=1, line_color='lightgrey',line_alpha=0.8)
r.segment(x0=c["longitude_Xi"], y0=c["latitude_Xi"], x1=c["longitude_Omega"],
          y1=c["latitude_Omega"], color="grey", line_width=1)

# Xi 
r.circle(np.array(dfTempX["longitude_Xi"]),np.array(dfTempX["latitude_Xi"]), size=5,fill_color='red', fill_alpha=.7, line_color='Crimson',line_alpha=0)
r.line(gbae[:,0],gbae[:,1], color='Crimson')
r.line(glus[:,0],glus[:,1], color='Crimson')
r.line(gtar1[:,0],gtar1[:,1], color='Crimson')
r.line(gtar2[:,0],gtar2[:,1], color='Crimson')
r.line([4.083,6.333,9,12], [37.667,39,39,37.25], line_alpha=0.8, color='Crimson')
r.line([5.333,9.333,9,9], [41.833,41.833,40.5,39], line_alpha=0.8, color='Crimson')
r.line([15.167,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color='Crimson')

# Omega
r.circle(np.array(dfTemp["longitude_Omega"]),np.array(dfTemp["latitude_Omega"]), size=5,fill_color='DodgerBlue', fill_alpha=0.8, line_color='DodgerBlue',line_alpha=0)
r.line(fbae[:,0],fbae[:,1], color='DodgerBlue')
r.line(flus[:,0],flus[:,1], color='DodgerBlue')
r.line(ftar1[:,0],ftar1[:,1], color='DodgerBlue')
r.line(ftar2[:,0],ftar2[:,1], color='DodgerBlue')
r.line([4.083,6.333,9,12], [37.667,39,39,37.25], line_alpha=0.8, color='DodgerBlue')
r.line([5.333,9.167,9,9], [41.833,41.333,40.167,39], line_alpha=0.8, color='DodgerBlue')
r.line([15,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color='DodgerBlue')

r.circle(np.array(IbEq["longitude_Xi"]),np.array(IbEq["latitude_Xi"]), size=5.5,fill_color='lightgrey', fill_alpha=1, line_color='lightgrey',line_alpha=1)

show(r);

In [24]:
g2=g.reset_index(level=['chap_ID','sec_ID','ID'])
g2=g2[g2.longitude_Xi.apply(lambda row: row not in [0])]
g2=g2[['ID','longitude_Xi','latitude_Xi']].dropna()

f2=f.reset_index(level=['chap_ID','sec_ID','ID'])
f2=f2[f2.longitude_Omega.apply(lambda row: row not in [0])]
f2=f2[['ID','longitude_Omega','latitude_Omega']].dropna()




i = pd.merge(left=f2, right=g2,how='inner')

IbEq2 = i[i["longitude_Xi"]==i["longitude_Omega"]]
IbEq2 = IbEq2[IbEq2["latitude_Xi"]==IbEq2["latitude_Omega"]]


In [25]:
r = figure(title='Iberian peninsula: Comparison between Xi (red) and Omega (blue)', width=1000, height=800, x_range=(1.5, 22), y_range=(35.5, 47), tools= [BoxZoomTool(), WheelZoomTool(), ResetTool()])

#r.circle(np.array(IbEq["longitude_Xi"]),np.array(IbEq["latitude_Xi"]), size=5.5,fill_color='lightgrey', fill_alpha=1, line_color='lightgrey',line_alpha=0.8)
#r.segment(x0=c["longitude_Xi"], y0=c["latitude_Xi"], x1=c["longitude_Omega"],
        #  y1=c["latitude_Omega"], color="grey", line_width=1)

# Xi 
r.circle(np.array(g["longitude_Xi"]),np.array(g["latitude_Xi"]), size=6,fill_color='red', fill_alpha=.7, line_color='Crimson',line_alpha=0)
r.line(gbae[:,0],gbae[:,1], color='Crimson',line_alpha=0.8)
r.line(glus[:,0],glus[:,1], color='Crimson',line_alpha=0.8)
r.line(gtar1[:,0],gtar1[:,1], color='Crimson',line_alpha=0.8)
r.line(gtar2[:,0],gtar2[:,1], color='Crimson',line_alpha=0.8)
#r.line([4.083,6.333,9,12], [37.667,39,39,37.25], line_alpha=0.8, color='Crimson')
#r.line([5.333,9.333,9,9], [41.833,41.833,40.5,39], line_alpha=0.8, color='Crimson')
r.line([15.167,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color='Crimson')

# Omega
r.circle(np.array(f["longitude_Omega"]),np.array(f["latitude_Omega"]), size=6,fill_color='blue', fill_alpha=0.8, line_color='blue',line_alpha=0)
r.line(fbae[:,0],fbae[:,1], color='blue',line_alpha=0.5)
r.line(flus[:,0],flus[:,1], color='blue',line_alpha=0.5)
r.line(ftar1[:,0],ftar1[:,1], color='blue',line_alpha=0.5)
r.line(ftar2[:,0],ftar2[:,1], color='blue',line_alpha=0.5)
#r.line([4.083,6.333,9,12], [37.667,39,39,37.25], line_alpha=0.8, color='DodgerBlue')
#r.line([5.333,9.167,9,9], [41.833,41.333,40.167,39], line_alpha=0.8, color='DodgerBlue')
r.line([15,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color='navy')

r.circle(np.array(IbEq2["longitude_Xi"]),np.array(IbEq2["latitude_Xi"]), size=6,fill_color='grey', fill_alpha=1, line_color='grey',line_alpha=1)

show(r);